## Name managing and reference file creation

Import necessary packages

In [48]:
#import packages
import os
import csv
import re
import platform
import pandas as pd
import numpy as np

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Root directory

In [50]:
if platform.system() == "Darwin":
    root_directory = '/Users/lorenzobarbiero/Documents/GitHub/CNN-Gait-recognition'
else:
    root_directory = '/content/drive/MyDrive/Human_Data/Progetto'

root_directory

'/content/drive/MyDrive/Human_Data/Progetto'

File name changing function

In [ ]:
def add_folder_to_filename(directory):
    # Iterate through all files and directories in the given directory
    for root, dirs, files in os.walk(directory):
        # Exclude .DS_Store files
        if '.DS_Store' in files:
            files.remove('.DS_Store')

        for file in files:
            # Generate the relative path of the file from the root directory
            relative_path = os.path.relpath(root, directory)

            # Replace '/' with '_' in the relative path


            # Construct the new filename by appending the modified relative path
            new_filename = os.path.join(relative_path, file)

            new_filename = new_filename.replace(os.sep, '_')
            new_filename = new_filename.replace("subject", "Sbj").replace("trial", "T").replace("pressure", "P").replace("skeleton", "S")

            # Construct the full paths for the old and new filenames
            old_path = os.path.join(root, file)
            new_path = os.path.join(root, new_filename)

            # Rename the file
            os.rename(old_path, new_path)
            print(f"Renamed '{old_path}' to '{new_path}'")


## DO NOT RUN NEXT CELL
Data needs to be in a "dataset" folder inside the root directory

In [ ]:
# # Specify the directory where you want to start renaming files
# directory = root_directory + '/Dataset'

# # Call the function to add folder names to filenames
# add_folder_to_filename(directory)

Renamed '/content/drive/MyDrive/Human_Data/Progetto/Dataset/subject9/antalgic/trial14/pressure.csv' to '/content/drive/MyDrive/Human_Data/Progetto/Dataset/subject9/antalgic/trial14/Sbj9_antalgic_T14_P.csv'
Renamed '/content/drive/MyDrive/Human_Data/Progetto/Dataset/subject9/antalgic/trial14/skeleton.csv' to '/content/drive/MyDrive/Human_Data/Progetto/Dataset/subject9/antalgic/trial14/Sbj9_antalgic_T14_S.csv'
Renamed '/content/drive/MyDrive/Human_Data/Progetto/Dataset/subject9/antalgic/trial20/pressure.csv' to '/content/drive/MyDrive/Human_Data/Progetto/Dataset/subject9/antalgic/trial20/Sbj9_antalgic_T20_P.csv'
Renamed '/content/drive/MyDrive/Human_Data/Progetto/Dataset/subject9/antalgic/trial20/skeleton.csv' to '/content/drive/MyDrive/Human_Data/Progetto/Dataset/subject9/antalgic/trial20/Sbj9_antalgic_T20_S.csv'
Renamed '/content/drive/MyDrive/Human_Data/Progetto/Dataset/subject9/antalgic/trial12/pressure.csv' to '/content/drive/MyDrive/Human_Data/Progetto/Dataset/subject9/antalgic/tri

Create reference file

In [51]:
label = 'antalgic'
LABELS = np.array(['antalgic', 'lurching', 'normal', 'steppage', 'stifflegged', 'trendelenburg'])
num = np.arange(6)
label_num = num[np.where(LABELS==label)]
label_num.item()

0

In [52]:
directory = root_directory + '/dataset'

In [53]:
gait = np.array(['antalgic', 'lurching', 'normal', 'steppage', 'stifflegged', 'trendelenburg'])
num_gait = np.arange(6)

def compile_csv(directory):
    # Open the CSV file in write mode
    with open(root_directory+'/reference_document.csv', 'w', newline='') as csvfile:
        # Define the CSV writer
        csv_writer = csv.writer(csvfile)
        # Write the header row
        csv_writer.writerow(['File_Name', 'Path_from_Root', 'Type', 'Label_str', 'Label'])

        # Iterate through all files and directories in the given directory
        for root, dirs, files in os.walk(directory):
            # Exclude .DS_Store files
            if '.DS_Store' in files:
                files.remove('.DS_Store')

            for file in files:
                # Extract information from the filename
                filename, ext = os.path.splitext(file)
                path_from_root = os.path.relpath(root, directory)
                type_ = "pressure" if filename[-1] == "P" else "skeleton"

                # Replace specific substrings in the filename
                label = filename.replace("Sbj","").replace("T","").replace("P","").replace("S","")

                # Remove non-alphanumeric characters and underscores
                label = re.sub(r'[^\w\s]|_', '', label)
                label = re.sub(r'\d', '', label)

                label_num = num_gait[np.where(gait==label)].item()

                # Write the information to the CSV file
                csv_writer.writerow([filename + '.csv', path_from_root+"/", type_, label, label_num])


# Call the function to compile CSV while renaming files
compile_csv(directory)

Import file as pandas dataframe

In [54]:
# Specify the path to the CSV file
csv_file = root_directory + '/reference_document.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file)

# Display the DataFrame
df

,File_Name,Path_from_Root,Type,Label_str,Label
0,Sbj9_antalgic_T2_S.csv,subject9/antalgic/trial2/,skeleton,antalgic,0
1,Sbj9_antalgic_T2_P.csv,subject9/antalgic/trial2/,pressure,antalgic,0
2,Sbj9_antalgic_T8_S.csv,subject9/antalgic/trial8/,skeleton,antalgic,0
3,Sbj9_antalgic_T8_P.csv,subject9/antalgic/trial8/,pressure,antalgic,0
4,Sbj9_antalgic_T14_P.csv,subject9/antalgic/trial14/,pressure,antalgic,0
...,...,...,...,...,...
2875,Sbj8_normal_T17_S.csv,subject8/normal/trial17/,skeleton,normal,2
2876,Sbj8_normal_T10_P.csv,subject8/normal/trial10/,pressure,normal,2
2877,Sbj8_normal_T10_S.csv,subject8/normal/trial10/,skeleton,normal,2
2878,Sbj8_normal_T6_P.csv,subject8/normal/trial6/,pressure,normal,2
